In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords

In [2]:
url = 'https://finance.naver.com/news/mainnews.nhn'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
headline = []
for index, link in enumerate(soup.findAll("a", href=re.compile("^/news/news_read.nhn."))):
    if 'href' in link.attrs and link.get('title') != None:
        headline.append(link.get('title'))

In [4]:
headline

['[단독] 홈쇼핑 마스크 완판 행진…정부, 공급 대책없이 유인책만',
 '이틀째 추가 확진자 0…의심환자는 939명으로 급증',
 "[주간증시 리뷰]'신종 코로나 충격' 회복 조짐 보인 코스피",
 '신풍제약 두자릿수 급락… 외인·기관 던지고, 개인 받았다',
 'KT, 올해 실적 회복기 진입 기대',
 '엠플러스, 연매출 1000억원 돌파…전년비 33.5% 상승',
 '[주목!e스몰캡]이노와이어리스, 5G 해외투자 사이클이 온다',
 '"여의도 신바람"…증시부진 뚫고 너도나도 \'최대실적\'',
 '엠플러스, 지난해 영업이익 31억원… 매출도 1000억원 돌파',
 "난기류 맞은 항공업계... 티웨이항공 홀로 ‘선방'",
 '[뉴욕마감] "中 성장률 0% 추락할 것"…다우 0.9% 뚝',
 '"삼성SDI, 올해 영업익 80% 증가 예상.. 목표가↑" DB금융투자',
 '[증시풍향계] 중국, 신종코로나 충격 극복할 경기부양 나설까',
 '[주간추천주]SK증권',
 '신풍제약 두자릿수 급락… 외인·기관 던지고, 개인 받았다',
 '[주간추천주]유안타증권',
 '[富를 키우는 투자지표]`저 세상 주식` 테슬라..거품 빠진 뒤에는',
 "[주간증시 리뷰]'신종 코로나 충격' 회복 조짐 보인 코스피"]

In [5]:
# removing functuation
def remove_punctuation(text):
    import string
    # replacing the puntuations with no space,
    # which in effect deletes the punctuation marks
    translator = str.maketrans(' ', ' ', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

In [6]:
# extract the stop words
# extracting the stopwords
sw = stopwords.words()
print(f'Number of stopwords: {len(sw)}')
stopwords_ko = [line.replace('\n', '') for line in open('./meta/stopwords_ko', 'r').readlines()]
print(f'Number of stopwords_ko: {len(stopwords_ko)}')
stopwords_total = stopwords_ko + sw
print(f'Number of stopwords_total: {len(stopwords_total)}')
# removing stopwords
def remove_stopwords(text):
    # removing the stop words and lowercasing the selected words
    text = [word for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

Number of stopwords: 6800
Number of stopwords_ko: 675
Number of stopwords_total: 7475


In [7]:
evaluation = pd.DataFrame()
evaluation['headline'] = headline
evaluation['tokenization'] = [re.sub(r'[^A-Za-z0-9가-힣一-龥]', ' ', i) for i in evaluation.headline]
evaluation['tokenization'] = evaluation['tokenization'].apply(remove_punctuation).apply(remove_stopwords)
evaluation['tokenization'] = [i.split(' ') for i in evaluation['tokenization']]

In [8]:
evaluation

,headline,tokenization
0,"[단독] 홈쇼핑 마스크 완판 행진…정부, 공급 대책없이 유인책만","[단독, 홈쇼핑, 마스크, 완판, 행진, 정부, 공급, 대책없이, 유인책만]"
1,이틀째 추가 확진자 0…의심환자는 939명으로 급증,"[이틀째, 추가, 확진자, 0, 의심환자는, 939명으로, 급증]"
2,[주간증시 리뷰]'신종 코로나 충격' 회복 조짐 보인 코스피,"[주간증시, 리뷰, 신종, 코로나, 충격, 회복, 조짐, 보인, 코스피]"
3,"신풍제약 두자릿수 급락… 외인·기관 던지고, 개인 받았다","[신풍제약, 두자릿수, 급락, 외인, 기관, 던지고, 개인, 받았다]"
4,"KT, 올해 실적 회복기 진입 기대","[KT, 올해, 실적, 회복기, 진입, 기대]"
5,"엠플러스, 연매출 1000억원 돌파…전년비 33.5% 상승","[엠플러스, 연매출, 1000억원, 돌파, 전년비, 33, 5, 상승]"
6,"[주목!e스몰캡]이노와이어리스, 5G 해외투자 사이클이 온다","[주목, e스몰캡, 이노와이어리스, 5G, 해외투자, 사이클이, 온다]"
7,"""여의도 신바람""…증시부진 뚫고 너도나도 '최대실적'","[여의도, 신바람, 증시부진, 뚫고, 너도나도, 최대실적]"
8,"엠플러스, 지난해 영업이익 31억원… 매출도 1000억원 돌파","[엠플러스, 지난해, 영업이익, 31억원, 매출도, 1000억원, 돌파]"
9,난기류 맞은 항공업계... 티웨이항공 홀로 ‘선방',"[난기류, 맞은, 항공업계, 티웨이항공, 홀로, 선방]"


In [9]:
negative = [line.replace('\n', '') for line in open('./meta/negative', 'r').readlines()]
positive = [line.replace('\n', '') for line in open('./meta/positive', 'r').readlines()]
sentimental = pd.read_csv('./meta/sentimental', sep='\t')

In [10]:
negative[:5], positive[:5]

(['가난', '가난뱅이', '가난살이', '가난살이하다', '가난설음'],
 ['(-;', '(^^)', '(^-^)', '(^^*', '(^_^)'])

In [11]:
sentimental.head(5)

,keyword,score
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1
